# OPTIMA Graduate Research Workshop
## On Benders decomposition

Andreas Ernst

Alysson M. Costa



## Toy example 

## 1. MIP implementation

In [2]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("A mixed integer program")


#variables

indices = [1, 2]
x = m.addVars(indices, name = 'x',)
y = m.addVars(indices, name = 'y', vtype = GRB.BINARY)

#objective
m.setObjective( x[1] - x[2] + y[1] + y[2] , GRB.MAXIMIZE)

#constraints
cons1 = m.addConstr( x[1] + x[2] + y[2] <= 2)
cons2 = m.addConstr( -x[1] - x[2] + y[1] <= -1)

#solve
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 6 nonzeros
Model fingerprint: 0x8ff96f06
Variable types: 2 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


## 2. Benders

### 2.1 Master template

In [ ]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model(#fill)
y = m.addVars(#fill)
z = m.addVar(#fill, ub=10000)
m.setObjective(#fill)

### 2.1 Dual subproblem template

In [ ]:
import gurobipy as gp
from gurobipy import GRB

s = gp.Model(#fill)
s.Params.InfUnbdInfo = 1 #allow to get dual ray
x = s.addVars(#fill)
cons1 = s.addConstr(#fill, name="cons1")
cons2 = s.addConstr(#fill, name="cons2")

### 2.2 Main loop template

In [ ]:
import gurobipy as gp
from gurobipy import GRB

#Benders loop
LB = -100000; UB = 100000

while(UB - LB >= 0.00001):
  #optimize Master
  m.optimize()
  UB = m.objVal

  #update the right hand side of the subproblem
  cons1.rhs = #fill
  cons2.rhs = #fill

  #optimize sub
  s.optimize()

  #generate cuts
  if s.status == 3:
    print("Infeasibility cut")
    # get the dual ray
    u1 = cons1.getAttr('FarkasDual') 
    u2 = cons2.getAttr('FarkasDual') 
    m.addConstr(#fill)  #add the new cut
  else:
    print("Feasibility cut")
    # get the dual ray
    u1 = cons1.getAttr('Pi')
    u2 = cons2.getAttr('Pi')
    m.addConstr( #fill)

    #fill (update the best lower bound if necessary).


# 3. Answers

### 3.1 Classic implementation

In [3]:
import gurobipy as gp
from gurobipy import GRB

#master
m = gp.Model()
y = m.addVars(2, vtype = GRB.BINARY, name="y")
z = m.addVar(name="z", ub=1000)
m.setObjective(y[0]  + y[1] + z, GRB.MAXIMIZE)
m.Params.OutputFlag= 0

#sub
s = gp.Model()
s.Params.InfUnbdInfo = 1
s.Params.OutputFlag= 0
x = s.addVars(2, name="x")
cons1 = s.addConstr(x[0] + x[1] <= 0, name="cons1")
cons2 = s.addConstr(-x[0] - x[1] <= 0, name="cons2")
s.setObjective(x[0] - x[1],GRB.MAXIMIZE)

#Benders loop

LB = -100000
UB = 100000

while(UB - LB >= 0.00001):

  #optimize Master
  m.optimize()
  UB = m.objVal

  #update sub
  cons1.rhs = 2 - y[1].x
  cons2.rhs = -1 - y[0].x

  #optimize sub
  s.optimize()

  #generate cuts
  if s.status == 3:
    print("Infeasibility constraint")
    u1 = cons1.getAttr('FarkasDual')
    u2 = cons2.getAttr('FarkasDual')
    expr = u1*(2 - y[1]) + u2*(-1  - y[0])
    m.addConstr(expr >= 0)
    print(expr, ">=", "0")
  else:
    print("Feasibility constraint")
    u1 = cons1.getAttr('Pi')
    u2 = cons2.getAttr('Pi')
    expr = u1*(2 - y[1]) + u2*(-1  - y[0])
    print(expr, ">=", "z")
    m.addConstr( expr >= z)
    if   y[0].x  + y[1].x  + s.objVal > LB:
      LB = y[0].x  + y[1].x  + s.objVal

  print(LB, " ", UB)

# # Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal solution value
print('Total profit: ', m.objVal)


Set parameter InfUnbdInfo to value 1
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
-100000   1002.0
Feasibility constraint
<gurobi.LinExpr: 2.0 + -1.0 y[1] + -0.0 y[0]> >= z
3.0   1001.0
Feasibility constraint
<gurobi.LinExpr: 2.0 + -1.0 y[1] + -0.0 y[0]> >= z
3.0   3.0
y[0] 1.0
z 2.0
Total profit:  3.0


### 3.2 Implementation with Lazy constraints

In [4]:
import gurobipy as gp
from gurobipy import GRB


#Initial Subproblem:
def initial_sub():
  s = gp.Model()
  s.Params.InfUnbdInfo = 1
  s.Params.OutputFlag= 0
  x = s.addVars(2, name="x")
  cons1 = s.addConstr(x[0] + x[1] <= 0, name="cons1")
  cons2 = s.addConstr(-x[0] -x[1] <= 0, name="cons2")
  s.setObjective(x[0] - x[1],GRB.MAXIMIZE)
  return s  

#Update subproblem
def update_subproblem(s,y1,y2):
  cons1 = s.getConstrByName("cons1")
  cons2 = s.getConstrByName("cons2")
  cons1.rhs = 2 - y2
  cons2.rhs = -1 - y1

def generate_cuts(model,where):
  if where == GRB.Callback.MIPSOL:
    valsy = model.cbGetSolution(model._y)
    print(valsy)
    update_subproblem(s,valsy[0],valsy[1])   
    s.optimize()    
    if s.status == 3:
      print("Infeasibility constraint")
      cons1 = s.getConstrByName("cons1")
      cons2 = s.getConstrByName("cons2")
      u1 = cons1.getAttr('FarkasDual')
      u2 = cons2.getAttr('FarkasDual')
      expr = u1*(2 - y[1]) + u2*(-1  - y[0])
      print(expr, ">=", "0")
      model.cbLazy( expr >= 0)
    else:
      print("Feasibility constraint")
      cons1 = s.getConstrByName("cons1")
      cons2 = s.getConstrByName("cons2")
      u1 = cons1.getAttr('Pi')
      u2 = cons2.getAttr('Pi')
      expr = u1*(2 - y[1]) + u2*(-1  - y[0])
      print(expr, ">=", "z")
      model.cbLazy( expr >= m._z)

s = initial_sub()
s.write("model.lp")

#master
m = gp.Model()
y = m.addVars(2, vtype = GRB.BINARY, name="y")
z = m.addVar(name="z", ub=1000)

m.setObjective(y[0]  + y[1] + z, GRB.MAXIMIZE)

m.Params.LazyConstraints = 1
m._y = y
m._z = z
m.Params.OutputFlag= 0
m.optimize(generate_cuts)

s.write("model.lp")


# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal solution value
print('Total profit: ', m.objVal)

Set parameter InfUnbdInfo to value 1
Set parameter LazyConstraints to value 1
{0: 1.0, 1: 1.0}
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
{0: 1.0, 1: 1.0}
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
{0: 1.0, 1: 1.0}
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
{0: 1.0, 1: 1.0}
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
{0: 1.0, 1: 1.0}
Infeasibility constraint
<gurobi.LinExpr: 1.0 + -1.0 y[1] + -1.0 y[0]> >= 0
{0: 1.0, 1: -0.0}
Feasibility constraint
<gurobi.LinExpr: 2.0 + -1.0 y[1] + -0.0 y[0]> >= z
{0: 1.0, 1: -0.0}
Feasibility constraint
<gurobi.LinExpr: 2.0 + -1.0 y[1] + -0.0 y[0]> >= z
y[0] 1.0
z 2.0
Total profit:  3.0
